## Listing 8.1 Reweighting a probability distribution to a different temperature


In [1]:
import numpy as np
def reweight_distribution(original_distribution, temperature=0.5):
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    return distribution / np.sum(distribution)





# Listing 8.2 Downloading and parsing the initial text file



In [2]:
import keras 
import numpy as np

path=keras.utils.get_file('nietzsche.txt',
origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

Corpus length: 600901


## Listing 8.3 Vectorizing sequences of characters


In [3]:
# You’ll extract sequences 
# of 60 characters.
maxlen=60
# You’ll sample a new sequence 
# every three characters.
step=3
#Holds the extracted sequences
sentences = []
#holds targets(the follow up charac)
next_chars = []

for i in range(0,len(text)-maxlen,step):
    sentences.append(text[i:i+maxlen])
    next_chars.append(text[i + maxlen])
    
print('Number of sequences:', len(sentences))

#List of unique characters in the corpus
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
# Dictionary that maps
# unique characters to their
# index in the list “chars”
char_indices = dict((char, chars.index(char)) for char in chars)

# One-hot encodes 
# the characters 
# into binary arrays
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
        y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 200281
Unique characters: 59
Vectorization...


C:\Users\HP\AppData\Local\Temp/ipykernel_5048/1407214211.py:30: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
C:\Users\HP\AppData\Local\Temp/ipykernel_5048/1407214211.py:31: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), len(chars)), dtype=np.bool)


## Building the network
This network is a single LSTM layer followed by a Dense classifier and softmax over all
 possible characters. But note that recurrent neural networks aren’t the only way to do
 sequence data generation; 1D convnets also have proven extremely successful at this
 task in recent times.

# Listing 8.4 Single-layer LSTM model for next-character prediction


In [4]:
from keras import layers
model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [5]:
# Because your targets are one-hot encoded, you’ll use categorical_crossentropy as
# the loss to train the model. 

optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

C:\Users\HP\anaconda3\lib\site-packages\keras\optimizers\legacy\rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


### Listing 8.4 Single-layer LSTM model for next-character prediction

In [6]:
from keras import layers,models

model = models.Sequential()

model.add(layers.LSTM(128,input_shape=(maxlen,len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

### Listing 8.5 Model compilation configuration


In [7]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

### Listing 8.6 Function to sample the next character given the model’s predictions


In [8]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

### Listing 8.7 Text-generation loop


In [9]:
import random
import sys

for epoch in range(1,5):
    print('epoch',epoch)
    model.fit(x, y, batch_size=128, epochs=1)
    start_index = random.randint(0, len(text)- maxlen- 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')
    
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

epoch 1
1565/1565 [==============================] - 206s 129ms/step - loss: 1.9619
--- Generating with seed: "
resisted and opposed in france: the fact remains, neverthel"
------ temperature: 0.2

resisted and opposed in france: the fact remains, neverthel------ temperature: 0.5

resisted and opposed in france: the fact remains, neverthel------ temperature: 1.0

resisted and opposed in france: the fact remains, neverthel------ temperature: 1.2

resisted and opposed in france: the fact remains, neverthelepoch 2
1565/1565 [==============================] - 302s 193ms/step - loss: 1.6189
--- Generating with seed: "ssion on him. the latter
obeys a superior and hence feels no"
------ temperature: 0.2
ssion on him. the latter
obeys a superior and hence feels no------ temperature: 0.5
ssion on him. the latter
obeys a superior and hence feels no------ temperature: 1.0
ssion on him. the latter
obeys a superior and hence feels no------ temperature: 1.2
ssion on him. the latter
obeys a superior 

In [11]:
for i in range(400):
    sampled = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(generated_text):
        sampled[0, t, char_indices[char]] = 1.
    preds = model.predict(sampled, verbose=0)[0]
    next_index = sample(preds, temperature)
    next_char = chars[next_index]
    generated_text += next_char
    generated_text = generated_text[1:]
    sys.stdout.write(next_char)

s vrilud give feel frating, e.


111

=take strength, a man,fricious
whichcest
onately almeted lyi opposedinessney ksouls.



thereby term
xome delight is porth-mority it.

 ngordous; evil
some contradund?--tuin,
how advancement.--why was ofges this masted to yities probabitured, e-civence, just hid" the sadming he that is "hagifent to puy of viroum to
locines withstprocietian callowing," to
cobor

### Implementing DeepDream in Keras
 You’ll start from a convnet pretrained on ImageNet. In Keras, many such convnets are
 available: VGG16, VGG19, Xception, ResNet50, and so on. You can implement Deep
Dream with any of them, but your convnet of choice will naturally affect your visualiza
tions, because different convnet architectures result in different learned features. The
 convnet used in the original DeepDream release was an Inception model, and in prac
tice Inception is known to produce nice-looking DeepDreams, so you’ll use the Incep
tion V3 model that comes with Keras.

### Listing 8.8 Loading the pretrained Inception V3 model

In [19]:
from keras.applications import inception_v3
from keras import backend as K
#  You won’t be training the model, so 
# this command disables all training
# specific operations
K.set_learning_phase(0)

# Builds the Inception V3 network, 
# without its convolutional base. 
# The model will be loaded with 
# pretrained ImageNet weights.

model=inception_v3.InceptionV3(weights='imagenet',include_top=False)

87910968/87910968 [==============================] - 40s 0us/step


### Listing 8.9 Setting up the DeepDream configuration

In [24]:
 layer_contributions = {
 'mixed2': 0.2,
 'mixed3': 3.,
 'mixed4': 2.,
 'mixed5': 1.5,
 }

### Listing 8.10 Defining the loss to be maximized

In [28]:
# Creates a dictionary that maps 
# layer names to layer instances
# Create a dictionary mapping layer names to layer objects
layer_dict = dict([(layer.name, layer) for layer in model.layers])

# Initialize the loss (as a float tensor, not a Keras variable)
loss = 0.0

# Loop through each important layer
for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]

    # Get the output of the layer
    activation = layer_dict[layer_name].output

    # Compute total number of elements in the activation tensor
    scaling = K.prod(K.cast(K.shape(activation), 'float32'))

    # Add the L2 norm (without border pixels) to the loss
    loss += coeff * K.sum(K.square(activation[:, 2:-2, 2:-2, :])) / scaling

### Listing 8.11 Gradient-ascent process

In [29]:
# This tensor holds the 
# generated image: the dream.
dream = model.input
# Computes the gradients of the 
# dream with regard to the loss
grads = K.gradients(loss, dream)[0]
#  Normalizes the gradients  
# (important trick)
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)


#  Sets up a Keras function 
# to retrieve the value of 
# the loss and gradients, 
# given an input image
outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
      outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
      grad_values = outs[1]
      return loss_value, grad_values